<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Библиотеки-и-первичное-прочтение-файла" data-toc-modified-id="Библиотеки-и-первичное-прочтение-файла-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Библиотеки и первичное прочтение файла</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBM</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

<div class="alert alert-info">
Привет! Меня зовут Никита Мишин и я буду твоим ревьюером по этому проекты.
Для простоты предлагаю общение на 'ты'. Буду предполагать, что ты не против:) 
Если более предпочтительно обращение на 'Вы', пиши, не стесняйся.
Также если будут возникать вопросы, аналогично, пиши:)

Предлагаю работать в известном тебе итеративном формате.
Итерация состоит в моей проверке твоего решения. 
После решения могут остаться какие-то недочеты, которые я попрошу тебя устранить, ты их исправляешь и я проверяю твои решения.
    Оставленные мною комментарии могут быть разного вида:
   
    - зеленый: элегантные решения, которые тебе стоит запомнит и в дальнейшем взять на вооружение:) 
    
    - желтый: сигнал о том, что есть некритичная вещь(не всегда ошибка), что нужно точно поправить в следующей работе, даже желательно в этой (полезно, в первую очередь, для тебя:) ).Также это рекомендации на будущее    

    - красный: недочет, который нужно исправить в этой работе, для того, чтобы она была принята
    
    - синий: полезная информация, доп ресурсы, "вопросы на подумать"

Также попрошу не удалять мои комментарии:) <a class="tocSkip">
</div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Библиотеки и первичное прочтение файла

In [1]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

загрузили библиотеки

In [2]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

**специально загрузил версию младше, со старшей вешался google colab на лемматизации**

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

загрузили файл, посмотрим ближе

In [4]:
display(df.head())
df.info()

In [5]:
df.sample(n = 5, random_state = 12345)


<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Еще иногда полезно использовать метод sample с зафиксированным random_state --- будет выдавать случайные строки датасета</div>

<div class="alert alert-info">
<h1>Комментарий учащегося <a class="tocSkip"></a></h1>

Спасибо за идею, взял на вооружение!

In [6]:
df['toxic'].value_counts()

Файл достаточно большой, содержит почти 160 000 строк, важно учитывать при тяжких задачах. При этом у нас классическая задача классификации текста. Есть признак text, который необходимо подготовить, а также у нас есть целевой признак toxic с большим перекосом значений (9 к 1), что может быть проблемой для достижения заданной f-меры. Для начала разберёмся со столбцом text.


<div class="alert alert-succes">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Да, налицо явный дисбаланс классов</div>

### Подготовка признаков

Подготовим функции и создадим новый столбец

In [7]:
# df['text'] = df['text'].values.astype('U')


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
А зачем нам юникод? У нас ведь чисто английский текст</div>

<div class="alert alert-warning">

---


<h1>Комментарий учащегося <a class="tocSkip"></a></h1>

Честно я и не понял зачем) пробовал и с ним и без, ничего не менялось, но раз в тренажере было с ним - сделал так :) убрал


<div class="alert alert-info">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
;)</div>

In [8]:
# m = Mystem()

# def clear_lemm_text(text):
#     filtro = re.sub(r'[^a-zA-Z]', ' ', text)
#     splito = filtro.split()
#     clearo = ' '.join(splito)
#     lemm_list = m.lemmatize(clearo)
#     lemm_text = ' '.join(lemm_list)
#     return lemm_text

In [9]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def clear_lemm_text_word(text):
    filtro = re.sub(r'[^a-zA-Z]', ' ', str(text))
    splito = filtro.split()
    # clearo = ' '.join(splito)
    lemm_list = list()
    for w in nltk.word_tokenize(' '.join(splito)):
        lemmm_uno = lemmatizer.lemmatize(w, get_wordnet_pos(w))
        lemm_list.append(lemmm_uno)
    lemm_text = ' '.join(lemm_list)
    return lemm_text

In [10]:
clifff = df['text'].head().apply(clear_lemm_text_word)
print(clifff)


<div class="alert alert-danger">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Мне кажется, тут надо в word_tokenize посылать текст, а ты туда лист слов посылаешь, а потом надо добавить:

    import nltk
    
    nltk.download('averaged_perceptron_tagger')
    
    
У меня вон шайтан машин внизу завелась))

</div>

<div class="alert alert-warning">

<h1>Комментарий учащегося v2 <a class="tocSkip"></a></h1>


Загрузил этот tagger, передал список - вроде все работает. Выполняется правда жуть как долго (20 минут) и модельки по качеству хуже, чем с mystem, но работает :) спасибо большое!


<div class="alert alert-success">
<h1>Комментарий ревьюера v3 <a class="tocSkip"></a></h1>
Ну, pymstem  вроде просто ничего не делал даже)</div>

In [11]:
# df['text'] = df['text'].astype(str)
df['ready_word_text'] = df['text'].apply(clear_lemm_text_word)
# df['text'].head().apply(clear_lemm_text_word)

In [12]:
df.head()



<div class="alert alert-danger">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
1) Если не ошибаюсь pymystem вроде для русского текста. Попробуй WordNetLemmatizeкr

2) Мы ведь должны применять лемматизацию отдельно к слову, а не предложению
</div>

<div class="alert alert-warning">
<h1>Комментарий учащегося <a class="tocSkip"></a></h1>


0. **Почему-то выдаёт ошибку на re.sub, при том, что я его не менял. Ошибка TypeError: expected string or bytes-like object, пытался по разному ему давать - всё равно не хочет. Сможешь помочь/натокнуть на мысль? я 4 часа потратил на гугление и эксперименты, но так и не понял, что у меня не так. Поэтому сделал проект рабочим и потом запустил эти 2 ячейки для ошибки**
1. Вроде бы pymystem может и в английский, но не уверен. Взял wordnet с pos (честно скорее копировал из источника, почему он в return у get_wordnet_pos берёт именно wordnet.NOUN - я не понял). Еще ругался на nltk.download('punkt'), я ему скачал - но я же перед этим очищаю текст на всё кроме букв, что ему не нравится?
2. Взял функцию ровно из тренажера (там тоже было на предложениях). Предполагаю, что он сам как-то делит внутри и лемматизирует слова. для wordnet добавил цикл на слова


<div class="alert alert-info">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Отписался выше)

1.2 нужно немного контекста пример ошибки или не работающего кода, так сложно понять))

</div>

In [13]:
# import nltk
# nltk.download('averaged_perceptron_tagger')z


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
За название переменных отдельный лайк, оценил)</div>

теперь пройдёмся по столбцу и создадим новый (очищенный и лемматизированны), готовый к использованию

In [14]:
# df['ready_text'] = df['text'].apply(clear_lemm_text)

теперь поделим на обучающую и тестовую части

In [15]:
features = df['ready_word_text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.25, random_state = 12345, stratify = target)


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Забыл указать параметр startify--- отвечает за то, чтобы исходное соотношение классов строго сохранялось в полученных выборках</div>

<div class="alert alert-warning">
<h1>Комментарий учащегося <a class="tocSkip"></a></h1>

Добавил, думал он учитывает по умолчанию


<div class="alert alert-info">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Круто, что добавил</div>

подготовка готова, перейдём к обучению


<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Круто было бы визуализировать слова из разных классов посредством WordCloud --- было бы очень наглядно</div>

<div class="alert alert-warning">
<h1>Комментарий учащегося <a class="tocSkip"></a></h1>

Для меня оказалось тяжеловато для изучения (не понял до конца как ему в итоге передать слова, когда это не одна строка), поставил себе в бэклог помучить, спасибо за идею! :)


<div class="alert alert-info">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Ок)</div>

## Обучение

Воспользуемся 2 моделями:

- Линейная регрессия
- LGBM

(Тут было двухчасовое мучение с Catboost и google colab, но сражение было проиграно)

Для начала создадим признаки с помощью TF-IDF

In [16]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2)) 

tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

print('Размер train:', tf_idf_train.shape)
print('Размер test:', tf_idf_test.shape)

Признаки созданы, перейдём к моделям



### Линейная регрессия

Для начала посмотрим на необходимость балансирования весов классов. Запустим 2 версии модели 

In [17]:
%%time

model_LR = LogisticRegression()
train_f1 = cross_val_score(model_LR, 
                      tf_idf_train, 
                      target_train, 
                      cv=3, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

In [18]:
%%time

model_LR_balanced = LogisticRegression(class_weight='balanced')
train_f1 = cross_val_score(model_LR_balanced, 
                      tf_idf_train, 
                      target_train, 
                      cv=3, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

Невооруженным взглядом видно, насколько баланс важен. Будем использовать только эту модель. На основе неё запустим перебор гиперпараметров


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Угу</div>

In [19]:
%%time

lr = LogisticRegression(max_iter = 1000, class_weight='balanced')
lr_params = {'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
             'C':[0.1, 1, 10]}

clf_lr = GridSearchCV(estimator = lr, param_grid = lr_params, scoring='f1', cv = 3)

clf_lr.fit(tf_idf_train, target_train)

print(clf_lr.best_params_)

Лучшие параметры найдены, посмотрим на получившийся F1 на тестовой выборке.

In [20]:
# clf_lr.fit(tf_idf_train, target_train)

lr_predict = clf_lr.predict(tf_idf_test)

print('F1 =', f1_score(target_test, lr_predict))

Несмотря на долгий подбор, модель Линейной регрессии справилась с заданным порогом метрики f1 в 0.75, показав результат в 0.790


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Бейзлайн успешно побит, поздравляю!</div>

### LGBM

Теперь попробуем LGBM

In [21]:
%%time
LGBM = LGBMClassifier(class_weight='balanced')
parametrs_LGBM = {'n_estimators' : [100],
              'depth' : [15, 5],
              'learning_rate' : [0.2],
              'random_state' : [12345]}
clf_LGBM = GridSearchCV(estimator = LGBM, param_grid = parametrs_LGBM, scoring='f1', cv = 3)

clf_LGBM.fit(tf_idf_train, target_train)

print(clf_LGBM.best_params_)

Лучшие параметры найдены, посмотрим какую метрику покаже модель на тесте.

In [22]:
lgbm3 = LGBMClassifier(n_estimators = 100,
              depth = 15,
            learning_rate = 0.2,
              random_state = 12345)

lgbm3.fit(tf_idf_train, target_train)

LGBM3_predict = lgbm3.predict(tf_idf_test)

print('F1 =', f1_score(target_test, LGBM3_predict))

LGBM показал отличный результат по F1 в 0.773, что также побило требуемый уровень в 0.75

## Выводы

**По результатам исследования подобрана лучшая модель - это модель Линейной Регрессии с гиперпараметрами: max_iter = 1000, class_weight='balanced', C = 10, solver = lbfgs! . Она показала F1 выше требуемого (0.790)**

*Важный комментарий: в целевом признаке наблюдается сильный дисбаланс, положительных признаков (токсичных комментариев) всего лишь 10% от общего числа! В связи с этим проверка на адекватность не проводилась, требуемый показатель F1 достаточно определяет целесообразность модели*

В процессе исследования мы провели лемматизацию и очищение текстов, а также провели создание признаков с помощью TF-IDF, после чего протестировали 2 модели с разными гиперпараметрами.


<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Круто было бы еще сводную табличку сделать)</div>

*P.S.Чесно говоря проект дико нервничал-делал, жесткий дедлайн 7-го числа и боюсь не успеть до него закрыть. Всё очень тяжелое и очень долго работает, впервые у меня не получилось сделать проект в самом тренажере. Я уже и с катбустом намучился и просто с обычными фитами пока перебирал гиперы вручную (просто афк по 10-20 минут). Начал делать берт, но у меня совсем начало всё виснуть даже на супернебольших выборках. Может я что-то делаю не так?) Сможете натолкнуть?)*


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Понял принял, запомнил, увижу на перепроврке --- сразу проерять сяду. 

Не переживай, тут осталось совсем чуть-чуть подправить)


С бертом --- тяжелая махина, хорошо бы свою машинку мощную иметь или времени иметь много) А так ссылочки могу покидать полезные для чтива на след итерации. Кидать?
</div>

<div class="alert alert-warning">
<h1>Комментарий учащегося <a class="tocSkip"></a></h1>

1. Мне тут сказали, что если я на первичную проверку закинул раньше 7-го, то всё ок и можно не так торопиться) Но спасибо тебе большое за понимание!
2. Да, покидай плз


<div class="alert alert-info">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Хах, ок)</div>

<div class="alert alert-info">
<h1> Комментарий ревьюера v2 <a class="tocSkip"></h1>
Полезное чтиво:
    
Для работы с текстами используют и другие подходы. 
Cейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). 
НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из 
классического ML тоже могут справляться. \
BERT тяжелее, да, но порой очень нужен. И существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers, они как раз работают нормально по скорости. 
Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.

Пачка полезных ссылок

1)<a href ='https://huggingface.co/transformers/model_doc/bert.html'>туть</a>

2)<a href='https://t.me/renat_alimbekov'>туть</a>

2) <a href='https://colah.github.io/posts/2015-08-Understanding-LSTMs'>Про LSTM</a>

3) <a href='https://web.stanford.edu/~jurafsky/slp3/10.pdf'> про энкодер-декодер модели, этеншены </a>

4) <a href = 'https://pytorch.org/tutorials/beginner/transformer_tutorial.html'> оф гайд по трансформеру от создателей pytorch </a>


5) <a href = 'https://transformer.huggingface.co/'> поболтать с трансформером </a>

6) Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множество реализованных
методов для трансформеров методов NLP

6') Отдельно выделю библиотеку TRAX, от гугла
    
7) курсы на <a href='https://stepik.org/course/54098'>степик</a> и  <a href='https://www.coursera.org/specializations/natural-language-processing'>coursera</a>

## Чек-лист проверки

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"></a></h2>

-Спасибо за качественно сделанный проект, было приятно проверять.

- Код написан хорошо 

- Соблюдена структура проекта 

- Оставил пару советов и комментариев

- Нужно немного доработать проект:
    
    - лемматизация

</div>

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Спасибо за качественную доработку проекта и удачи в следующих проектах!</div>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Библиотеки-и-первичное-прочтение-файла" data-toc-modified-id="Библиотеки-и-первичное-прочтение-файла-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Библиотеки и первичное прочтение файла</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBM</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>